# Implementing and training attention mechanism

In [10]:
import random
import tensorflow as tf
import pandas as pd
# Korean NLP package (Open Korean Tweet Parser)
from konlpy.tag import Okt

In [5]:
EPOCHS = 200
NUM_WORDS = 2000

## Encoder

In [6]:
class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        self.lstm = tf.keras.layers.LSTM(512, return_state=True)
        
    def call(self, x, training=False, mask=None):
        x = self.emb(x)
        _, h, c = self.lstm(x)
        return h, c

## Decoder

In [7]:
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder, self).__init__()
        # Adding layers for shifted output
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        # decoder need to store whole state from lstm so return_sequences should be True
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(NUM_WORDS, activation='softmax')
        
    def call(self, inputs, training=False, mask=None):
        x, h, c = inputs
        x = self.emb(x)
        x, h, c = self.lstm(x, initial_state=[h, c])
        return self.dense(x), h, c

## Seq2Seq

In [8]:
class Seq2seq(tf.keras.Model):
    def __init__(self, sos, eos):
        super(Seq2seq, self).__init__()
        self.enc = Encoder()
        self.dec = Decoder()
        self.sos = sos
        self.eos = eos
        
    def call(self, inputs, training=False, mask=None):
        # When training, input is just shifted, so the process is very simple
        if training:
            x, y = inputs
            h, c = self.enc(x)
            y, _, _ = self.dec((y, h, c))
            return y
        else:
            x = inputs
            h, c = self.enc(x)
            y = tf.convert_to_tensor(self.sos)
            y = tf.reshape(y, (1, 1))
            
            seq = tf.TensorArray(tf.int32, 64)
            
            # Through tf.range, we can iterate it with autoGraph
            for idx in tf.range(64):
                y, h, c = self.dec([y, h, c])
                
                # To convert sparse type
                y = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32)
                
                # To express batch shape
                y = tf.reshape(y, (1, 1))
                seq = seq.write(idx, y)
                
                if y == self.eos:
                    break
                
            return tf.reshape(seq.stack(), (1, 64))

## Training Loop

In [9]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    output_labels = labels[:, 1:]
    shifted_labels = labels[:, :-1]
    
    with tf.GradientTape() as tape:
        predictions = model([inputs, shifted_labels], training=True)
        loss = loss_object(output_labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(output_labels, predictions)
    
@tf.function
def test_step(model, inputs):
    return model(inputs, training=False)

## Preparing Dataset

In [12]:
okt = Okt()

chatbot_data = pd.read_csv('./dataset/ChatbotData.csv')
chatbot_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [18]:
questions = [' '.join(okt.morphs(line)) for line in chatbot_data['Q']]
answers = [' '.join(okt.morphs(line)) for line in chatbot_data['A']]

In [24]:
answers_t = ['\t ' + line for line in answers]

In [25]:
answers_t

['\t 하루 가 또 가네요 .',
 '\t 위로 해 드립니다 .',
 '\t 여행 은 언제나 좋죠 .',
 '\t 여행 은 언제나 좋죠 .',
 '\t 눈살 이 찌푸려지죠 .',
 '\t 다시 새로 사는 게 마음 편해요 .',
 '\t 다시 새로 사는 게 마음 편해요 .',
 '\t 잘 모르고 있을 수도 있어요 .',
 '\t 시간 을 정 하고 해보세요 .',
 '\t 시간 을 정 하고 해보세요 .',
 '\t 자랑 하는 자리 니까 요 .',
 '\t 그 사람 도 그럴 거 예요 .',
 '\t 그 사람 도 그럴 거 예요 .',
 '\t 혼자 를 즐기세요 .',
 '\t 돈 은 다시 들어올 거 예요 .',
 '\t 땀 을 식혀주세요 .',
 '\t 어서 잊고 새 출발 하세요 .',
 '\t 빨리 집 에 돌아가서 끄고 나오세요 .',
 '\t 빨리 집 에 돌아가서 끄고 나오세요 .',
 '\t 다음 달 에는 더 절약 해봐요 .',
 '\t 따뜻하게 사세요 !',
 '\t 다음 달 에는 더 절약 해봐요 .',
 '\t 가장 확실한 시간 은 오늘이 에요 . 어제 와 내일 을 놓고 고민 하느라 시간 을 낭비하지 마세요 .',
 '\t 온 가족 이 모두 마음 에 드는 곳 으로 가보세요 .',
 '\t 온 가족 이 모두 마음 에 드는 곳 으로 가보세요 .',
 '\t 온 가족 이 모두 마음 에 드는 곳 으로 가보세요 .',
 '\t 저 를 만들어 준 사람 을 부모님 , 저 랑 이야기 해 주는 사람 을 친구 로 생각 하고 있어요',
 '\t 저 를 만들어 준 사람 을 부모님 , 저 랑 이야기 해 주는 사람 을 친구 로 생각 하고 있어요',
 '\t 더 가까워질 기회 가 되겠네요 .',
 '\t 저 도 요 .',
 '\t 다 들 바빠서 이야기 할 시간 이 부족했나 봐요 .',
 '\t 다 들 바빠서 이야기 할 시간 이 부족했나 봐요 .',
 '\t 온 가족 이 모두 마음 에 드는 곳 으로 가보세요 .',
 '\t 좋은 생각 이에요 .',
 '\t 더 가까워질 기회 가 되겠네

In [26]:
num_sample = len(questions)

In [27]:
perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)

In [28]:
train_q = []
train_a = []
test_q = []
test_a = []

for idx, qna in enumerate(zip(questions, answers_t)):
    q, a = qna
    if perm[idx] > num_sample // 5:
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)
    

In [31]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,
                                                 filters='!"#$%()*+,-./:;<=>?@[\\]^_`{|}~')

In [32]:
tokenizer.fit_on_texts(train_q + train_a)